In [133]:
import json
import re

def tokenize(s):
    # Extract tokens: parentheses or sequences of non-whitespace, non-parenthesis characters.
    tokens = re.findall(r'\(|\)|[^\s()]+', s)
    return tokens

def tokens_to_ints(tokens, vocab):
    # map the tokens to integers from vocab
    # if the token is not in the vocab, use the index of the unknown token
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]

def parse_tokens(tokens):
    # Parse tokens into a nested list structure
    stack = []
    current_list = []
    for token in tokens:
        if token == '(':
            stack.append(current_list)
            current_list = []
        elif token == ')':
            finished = current_list
            current_list = stack.pop()
            current_list.append(finished)
        else:
            current_list.append(token)
    return current_list

def normalize_structure(tree):
    if not isinstance(tree, list):
        return None

    def is_key(token):
        return token in [
            "ORDER", "PIZZAORDER", "DRINKORDER", "NUMBER", "SIZE", "STYLE", "TOPPING",
            "COMPLEX_TOPPING", "QUANTITY", "VOLUME", "DRINKTYPE", "CONTAINERTYPE", "NOT"
        ]

    # Clean the list by keeping sublists and tokens as-is for further analysis
    cleaned = []
    for el in tree:
        cleaned.append(el)

    if len(cleaned) > 0 and isinstance(cleaned[0], str) and is_key(cleaned[0]):
        key = cleaned[0]
        if key == "ORDER":
            pizzaorders = []
            drinkorders = []
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict):
                    if "PIZZAORDER" in node:
                        if isinstance(node["PIZZAORDER"], list):
                            pizzaorders.extend(node["PIZZAORDER"])
                        else:
                            pizzaorders.append(node["PIZZAORDER"])
                    if "DRINKORDER" in node:
                        if isinstance(node["DRINKORDER"], list):
                            drinkorders.extend(node["DRINKORDER"])
                        else:
                            drinkorders.append(node["DRINKORDER"])
                    if node.get("TYPE") == "PIZZAORDER":
                        pizzaorders.append(node)
                    if node.get("TYPE") == "DRINKORDER":
                        drinkorders.append(node)
            result = {}
            if pizzaorders:
                result["PIZZAORDER"] = pizzaorders
            if drinkorders:
                result["DRINKORDER"] = drinkorders
            if result:
                return {"ORDER": result}
            else:
                return {}

        elif key == "PIZZAORDER":
            number = None
            size = None
            style = None
            toppings = []
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict):
                    t = node.get("TYPE")
                    if t == "NUMBER":
                        number = node["VALUE"]
                    elif t == "SIZE":
                        size = node["VALUE"]
                    elif t == "STYLE":
                        style = node["VALUE"]
                    elif t == "TOPPING":
                        toppings.append(node)
            result = {}
            if number is not None:
                result["NUMBER"] = number
            if size is not None:
                result["SIZE"] = size
            if style is not None:
                result["STYLE"] = style
            if toppings:
                result["AllTopping"] = toppings
            # Mark type internally, will remove later
            result["TYPE"] = "PIZZAORDER"
            return result

        elif key == "DRINKORDER":
            number = None
            volume = None
            drinktype = None
            containertype = None
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict):
                    t = node.get("TYPE")
                    if t == "NUMBER":
                        number = node["VALUE"]
                    elif t == "VOLUME" or t == "SIZE":
                        volume = node["VALUE"]
                    elif t == "DRINKTYPE":
                        drinktype = node["VALUE"]
                    elif t == "CONTAINERTYPE":
                        containertype = node["VALUE"]
            result = {}
            if number is not None:
                result["NUMBER"] = number
            if volume is not None:
                result["SIZE"] = volume
            if drinktype is not None:
                result["DRINKTYPE"] = drinktype
            if containertype is not None:
                result["CONTAINERTYPE"] = containertype
            result["TYPE"] = "DRINKORDER"
            return result

        elif key in ["NUMBER","SIZE","STYLE","VOLUME","DRINKTYPE","CONTAINERTYPE","QUANTITY"]:
            values = []
            for el in cleaned[1:]:
                if isinstance(el, str):
                    values.append(el)
            value_str = " ".join(values).strip()
            return {
                "TYPE": key,
                "VALUE": value_str
            }

        elif key == "TOPPING":
            values = []
            for el in cleaned[1:]:
                if isinstance(el, str):
                    values.append(el)
            topping_str = " ".join(values).strip()
            return {
                "TYPE": "TOPPING",
                "NOT": False,
                "Quantity": None,
                "Topping": topping_str
            }
            
        elif key == "QUANTITY":
            # look for the first TOPPING key after you and assign the quantity to it
            quantity = None
            topping = None
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict):
                    t = node.get("TYPE")
                    if t == "QUANTITY":
                        quantity = node["VALUE"]
                    elif t == "TOPPING":
                        topping = node["Topping"]

        elif key == "COMPLEX_TOPPING":
            quantity = None
            topping = None
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict):
                    t = node.get("TYPE")
                    if t == "QUANTITY":
                        quantity = node["VALUE"]
                    elif t == "TOPPING":
                        topping = node["Topping"]
            return {
                "TYPE": "TOPPING",
                "NOT": False,
                "Quantity": quantity,
                "Topping": topping
            }

        elif key == "NOT":
            for sub in cleaned[1:]:
                node = normalize_structure(sub)
                if isinstance(node, dict) and node.get("TYPE") == "TOPPING":
                    node["NOT"] = True
                    if "Quantity" not in node:
                        node["Quantity"] = None
                    return node
            return None

    else:
        # Try to parse sublists and combine orders found
        combined_order = {"PIZZAORDER": [], "DRINKORDER": []}
        found_order = False

        for el in cleaned:
            node = normalize_structure(el)
            if isinstance(node, dict):
                if "ORDER" in node:
                    found_order = True
                    order_node = node["ORDER"]
                    if "PIZZAORDER" in order_node:
                        combined_order["PIZZAORDER"].extend(order_node["PIZZAORDER"])
                    if "DRINKORDER" in order_node:
                        combined_order["DRINKORDER"].extend(order_node["DRINKORDER"])
                elif node.get("TYPE") == "PIZZAORDER":
                    found_order = True
                    combined_order["PIZZAORDER"].append(node)
                elif node.get("TYPE") == "DRINKORDER":
                    found_order = True
                    combined_order["DRINKORDER"].append(node)

        if found_order:
            final = {}
            if combined_order["PIZZAORDER"]:
                final["PIZZAORDER"] = combined_order["PIZZAORDER"]
            if combined_order["DRINKORDER"]:
                final["DRINKORDER"] = combined_order["DRINKORDER"]
            return {"ORDER": final} if final else {}

        return None

def remove_type_keys(obj):
    # Recursively remove "TYPE" keys from all dictionaries
    if isinstance(obj, dict):
        obj.pop("TYPE", None)
        for k, v in obj.items():
            remove_type_keys(v)
    elif isinstance(obj, list):
        for item in obj:
            remove_type_keys(item)


def preprocess(text):
    tokens = tokenize(text)
    parsed = parse_tokens(tokens)
    result = normalize_structure(parsed)
    remove_type_keys(result)
    return result

input_str = "(ORDER potato potato junior (PIZZAORDER (NUMBER one) (SIZE large) (STYLE thin crust) (TOPPING hot cheese) (TOPPING pepperoni) ) (PIZZAORDER (NUMBER two) (SIZE medium) (STYLE deep dish) (NOT (TOPPING mushrooms) ) (NOT (COMPLEX_TOPPING (QUANTITY extra) (TOPPING olives) ) ) ) (DRINKORDER (NUMBER five) (VOLUME one liter) (DRINKTYPE lemon ice tea) (CONTAINERTYPE bottles)) (DRINKORDER (NUMBER three) (VOLUME two liters) (DRINKTYPE cola) (CONTAINERTYPE cans)) (DRINKORDER (NUMBER three) (VOLUME two liters) (DRINKTYPE cola) (CONTAINERTYPE cans) ) )"

result = preprocess(input_str)

print(json.dumps(result, indent=2))


{
  "ORDER": {
    "PIZZAORDER": [
      {
        "NUMBER": "one",
        "SIZE": "large",
        "STYLE": "thin crust",
        "AllTopping": [
          {
            "NOT": false,
            "Quantity": null,
            "Topping": "hot cheese"
          },
          {
            "NOT": false,
            "Quantity": null,
            "Topping": "pepperoni"
          }
        ]
      },
      {
        "NUMBER": "two",
        "SIZE": "medium",
        "STYLE": "deep dish",
        "AllTopping": [
          {
            "NOT": true,
            "Quantity": null,
            "Topping": "mushrooms"
          },
          {
            "NOT": true,
            "Quantity": "extra",
            "Topping": "olives"
          }
        ]
      }
    ],
    "DRINKORDER": [
      {
        "NUMBER": "five",
        "SIZE": "one liter",
        "DRINKTYPE": "lemon ice tea",
        "CONTAINERTYPE": "bottles"
      },
      {
        "NUMBER": "three",
        "SIZE": "two liters",
    

In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def load_vocab():
    # loads the vocab from the text file "vocab.txt" and then swaps the key value pairs
    with open("../dataset2/vocab.txt", "r") as f:
        vocab = f.readlines()
    # remove any commas and single quotes
    vocab = [v.replace(",", "").replace("'", "") for v in vocab]
    vocab = {v.split(":")[0].strip():int(v.split(":")[1].strip()) for v in vocab}
    return vocab

vocab = load_vocab()
print(vocab)

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
        
        # Bidirectional LSTM
        self.bilstm_1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout,
        )

        # Fully connected layers
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)


    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)

        # BiLSTM layer
        lstm_out, _ = self.bilstm_1(embedded)

        output = self.fc2(lstm_out)
        return F.log_softmax(output, dim=-1)

input_dim = len(vocab) 
embedding_dim = 128  # len(vocab)
hidden_dim = 128  # Hidden state size for LSTM          first was 256 for batch norm 150
output_dim1 = 6  # Number of output classes
output_dim2 = 40  # Number of output classes
num_layers = 2  # Number of BiLSTM layers
dropout = 0.3  # Dropout probability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim1, num_layers, dropout).to(device)
model_2 = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim2, num_layers, dropout).to(device)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

# load the 2 models
load_model(model_1, "../weights/Bilstm_order_sequence.pt")
load_model(model_2, "../weights/Bilstm_model2.pt")

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'large': 4, 'pie': 5, 'with': 6, 'green': 7, 'pepper': 8, 'and': 9, 'extra': 10, 'ham': 11, 'party': 12, 'size': 13, 'stuffed': 14, 'crust': 15, 'artichokes': 16, 'mushroom': 17, 'i': 18, 'want': 19, 'one': 20, 'regular': 21, 'pizza': 22, 'without': 23, 'any': 24, 'basil': 25, 'a': 26, 'american': 27, 'cheese': 28, 'little': 29, 'bit': 30, 'of': 31, 'sausage': 32, '"d"': 33, 'like': 34, 'sized': 35, 'high': 36, 'rise': 37, 'dough': 38, 'lot': 39, 'banana': 40, 'chicken': 41, 'black': 42, 'olives': 43, 'sauce': 44, 'broccoli': 45, 'peperonni': 46, 'italian': 47, 'can': 48, 'have': 49, 'flatbread': 50, 'style': 51, 'lunch': 52, '-': 53, 'blue': 54, 'need': 55, 'caramelized': 56, 'onions': 57, 'combination': 58, 'eggplant': 59, 'pecorino': 60, 'New': 61, 'York': 62, 'artichoke': 63, 'spinach': 64, 'Neapolitan': 65, 'bacon': 66, 'tofu': 67, 'grilled': 68, 'mozzarella': 69, 'vegan': 70, 'pepperoni': 71, 'no': 72, 'feta': 73, 'yellow': 74, 'pe

C:\Users\yousi\AppData\Local\Temp\ipykernel_4128\1166127925.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


BiLSTMModel(
  (embedding): Embedding(607, 128, padding_idx=0)
  (bilstm_1): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc2): Linear(in_features=256, out_features=40, bias=True)
)

In [ ]:
import json
import torch

# Label maps for the models
MODEL_1_LABEL_MAP = {
    "B-PIZZAORDER": 1,
    "I-PIZZAORDER": 2,
    "B-DRINKORDER": 3,
    "I-DRINKORDER": 4,
    "O": 5
}

MODEL_2_LABEL_MAP = {
    'B-DRINKTYPE': 1, 'I-DRINKTYPE': 2,
    'B-SIZE': 3, 'I-SIZE': 4,
    'B-NUMBER': 5, 'I-NUMBER': 6,
    'B-CONTAINERTYPE': 7, 'I-CONTAINERTYPE': 8,
    'B-COMPLEX_TOPPING': 9, 'I-COMPLEX_TOPPING': 10,
    'B-TOPPING': 11, 'I-TOPPING': 12,
    'B-NEG_TOPPING': 13, 'I-NEG_TOPPING': 14,
    'B-NEG_STYLE': 15, 'I-NEG_STYLE': 16,
    'B-STYLE': 17, 'I-STYLE': 18,
    'B-QUANTITY': 19, 'I-QUANTITY': 20,
    'O': 21
}

# Function to apply both models and get the TOP_DECOUPLED format
def process_entry(entry):
    src_text = entry["train.SRC"]
    true_top = entry["train.TOP"]

    # Tokenize and preprocess the input text
    tokens = tokenize(src_text)
    # Convert tokens to integers
    tokens = tokens_to_ints(tokens, vocab)
    # Convert to tensor
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    # Get predictions from the first model
    model_1_output = model_1(tokens)
    first_model_labels = model_1_output.argmax(dim=-1).squeeze(0).tolist()  # Ensure list of labels

    # Get predictions from the second model
    model_2_output = model_2(tokens)
    second_model_labels = model_2_output.argmax(dim=-1).squeeze(0).tolist()  # Ensure list of labels

    # Generate TOP_DECOUPLED output
    top_decoupled = generate_top_decoupled(src_text, first_model_labels, second_model_labels)

    # Preprocess TOP to JSON format
    predicted_json = preprocess(top_decoupled)
    true_json = preprocess(true_top)
    
    with open("predicted_true.json", "a") as f:
        f.write(f"true top: {true_top}\n") 
    
    write_comparison_file(predicted_json, true_json, top_decoupled)

    # Compare the predicted JSON with the ground truth JSON
    return predicted_json == true_json

def generate_top_decoupled(text, first_labels, second_labels):
    words = text.split()
    first_labels = first_labels[:len(words)]
    second_labels = second_labels[:len(words)]
    
    # Debugging output
    with open("predicted_true.json", "a") as f:
        f.write(str(words) + "\n")
        f.write(str([next(k for k, v in MODEL_1_LABEL_MAP.items() if v == l) for l in first_labels]) + "\n")
        f.write(str([next(k for k, v in MODEL_2_LABEL_MAP.items() if v == l) for l in second_labels]) + "\n\n") 
    
    
    result = ["(ORDER"]
    current_order_type = None
    current_group = None
    open_groups = []  # To keep track of open groups for proper closing
    
    def negative_close_group():
        if current_group is not None:
            result.append(")")
            if open_groups:
                if open_groups:                 
                    open_groups.pop()
            # if we are closing a negative topping and the top group is a complex topping
            # we close that as well
            if current_group["type"] == "NEG_TOPPING" or current_group["type"] == "TOPPING" and open_groups[-1] == "COMPLEX_TOPPING":
                result.append(")")
                if open_groups:                 
                    open_groups.pop()
                current_group = None
            
    def positive_close_group():
        if current_group is not None:
            result.append(")")  # Close the previous group
            # since this is positive, if the current top group is a not group close it as well
            if current_group["type"] == "NEG_TOPPING" or current_group["type"] == "NEG_STYLE":
                result.append(")")
            if open_groups:                 
                open_groups.pop()
            # if we are closing a negative topping and the top group is a complex topping
            # we close that as well
            if current_group["type"] == "NEG_TOPPING" or current_group["type"] == "TOPPING" and open_groups[-1] == "COMPLEX_TOPPING":
                result.append(")")
                if open_groups:                 
                    open_groups.pop()
                current_group = None

    for i, (word, first_label, second_label) in enumerate(zip(words, first_labels, second_labels)):
        # Handle the first labels (ORDER type: PIZZAORDER, DRINKORDER)
        if first_label in [MODEL_1_LABEL_MAP["B-PIZZAORDER"], MODEL_1_LABEL_MAP["B-DRINKORDER"]]:
            if current_order_type is not None:
                result.append(")")  # Close the previous order
                if open_groups:                 
                    open_groups.pop()  # Remove from open_groups stack
            current_order_type = "PIZZAORDER" if first_label == MODEL_1_LABEL_MAP["B-PIZZAORDER"] else "DRINKORDER"
            result.append(f"({current_order_type}")
            open_groups.append(current_order_type)

        elif first_label == MODEL_1_LABEL_MAP["O"] and current_order_type is not None:
            result.append(")")  # Close the current order
            if open_groups:                 
                open_groups.pop()
            current_order_type = None
            
        elif first_label == MODEL_1_LABEL_MAP["O"] and current_order_type is None:
            continue  # Skip the word if it's not part of an order

        # Handle the second labels (attributes like NUMBER, SIZE, TOPPING, etc.)
        if second_label != MODEL_2_LABEL_MAP["O"]:
            second_label_key = next(
                (key for key, value in MODEL_2_LABEL_MAP.items() if value == second_label), None
            )
            if not second_label_key:
                print(f"Warning: Unexpected label {second_label} encountered for word '{word}'. Skipping.")
                continue

            label_type = second_label_key.split("-")[-1]
            if label_type not in ["NEG_TOPPING", "NEG_STYLE", "QUANTITY"]:
                if second_label_key.startswith("B-"):
                    # Close the previous group if there is one
                    if current_group:
                        positive_close_group()
                        
                    current_group = {"type": label_type, "content": [word]}
                    result.append(f"({label_type} {word}")
                    open_groups.append(label_type)

                elif second_label_key.startswith("I-") and current_group and current_group["type"] == label_type:
                    current_group["content"].append(word)
                    result[-1] += f" {word}"  # Append to the last open group

                elif second_label_key.startswith("I-") and (not current_group or current_group["type"] != label_type):
                    print(f"Warning: I- tag '{label_type}' for word '{word}' without preceding B- tag. Converting to B-.")
                    # Close the previous group if there is one
                    if current_group:
                        positive_close_group()
                    current_group = {"type": label_type, "content": [word]}
                    result.append(f"({label_type} {word}")
                    open_groups.append(label_type)
                    
            # special handling for QUANTITY (COMPLEX_TOPPING)
            elif label_type == "QUANTITY":
                if second_label_key.startswith("B-"):
                    if current_group:
                        positive_close_group()
                    result.append(f"(COMPLEX_TOPPING (QUANTITY {word}")
                    current_group = {"type": label_type, "content": [word]}
                elif second_label_key.startswith("I-") and current_group and current_group["type"] == label_type:
                    current_group["content"].append(word)
                    result[-1] += f" {word}"
                elif second_label_key.startswith("I-") and (not current_group or current_group["type"] != label_type):
                    if current_group:
                        positive_close_group()
                    print(f"Warning: I- tag '{label_type}' for word '{word}' without preceding B- tag. Converting to B-.")
                    result.append(f"(COMPLEX_TOPPING (QUANTITY {word}")
                    current_group = {"type": label_type, "content": [word]}
                # we add this here and then if we pop a topping and find COMPLEX_TOPPING
                # at the top we will close it
                open_groups.append("COMPLEX_TOPPING")
                open_groups.append(label_type)
            
            # Special handling for NEG_TOPPING and NEG_STYLE
            else:
                if second_label_key.startswith("B-"):
                    if current_group:
                        negative_close_group()
                    result.append(f"(NOT ({'TOPPING' if label_type == 'NEG_TOPPING' else 'STYLE'} {word}")
                    current_group = {"type": label_type, "content": [word]}
                    open_groups.append(label_type)
                elif second_label_key.startswith("I-") and current_group and current_group["type"] == label_type:
                    current_group["content"].append(word)
                    result[-1] += f" {word}"  # Append to the last open group
                elif second_label_key.startswith("I-") and (not current_group or current_group["type"] != label_type):
                     # Close the previous group if there is one
                    if current_group:
                        negative_close_group()
                    print(f"Warning: I- tag '{label_type}' for word '{word}' without preceding B- tag. Converting to B-.")
                    result.append(f"(NOT ({'TOPPING' if label_type == 'NEG_TOPPING' else 'STYLE'} {word}")
                    current_group = {"type": label_type, "content": [word]}
                    open_groups.append(label_type)
                    
        # Handle O labels
        else:
            if current_group:
                result.append(")")  # Close the current group
                if open_groups:                 
                    open_groups.pop()
            current_group = None

    # Close any remaining open groups
    while open_groups:
        result.append(")")
        
        if open_groups:                 
            open_groups.pop()

    result.append(")")  # Close the overall ORDER group
    return " ".join(result)

# Updated JSON comparison file formatting
def write_comparison_file(predicted_json, true_json, top_decoupled):
    with open("predicted_true.json", "a") as f:
        # write top decoupled 
        f.write(f"TOP_DECOUPLED: {top_decoupled}\n\n")
        # Format both JSONs side by side in a neat way
        predicted_str = json.dumps(predicted_json, indent=2)
        true_str = json.dumps(true_json, indent=2)

        max_width = max(len(line) for line in predicted_str.splitlines()) + 5
        max_lines = max(len(predicted_str.splitlines()), len(true_str.splitlines()))
        
        predicted_lines = predicted_str.splitlines()
        true_lines = true_str.splitlines()

        f.write(f"{'-' * (max_width * 2)}\n")
        f.write(f"{'Predicted'.center(max_width)}{'True'.center(max_width)}\n")
        f.write(f"{'-' * (max_width * 2)}\n")
        
        for i in range(max_lines):
            left = predicted_lines[i] if i < len(predicted_lines) else ""
            right = true_lines[i] if i < len(true_lines) else ""
            f.write(f"{left:<{max_width}}{right:<{max_width}}\n")
        
        f.write(f"{'-' * (max_width * 2)}\n")
        f.write(f"Match: {predicted_json == true_json}\n")
        f.write(f"{'-' * (max_width * 2)}\n\n")

# Main function to process the dataset
def process_dataset(input_file):
    correct = 0
    total = 0
    # Clear the debug file
    open("predicted_true.json", "w").close()

    with open(input_file, "r") as infile:
        for line in infile:
            entry = json.loads(line)
            if process_entry(entry):
                correct += 1
            total += 1

    accuracy = correct / total * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")
    return accuracy

process_dataset("../dataset2/test_train.json")


UnboundLocalError: cannot access local variable 'current_group' where it is not associated with a value